In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os 
import weaviate
from weaviate.classes.init import Auth
from sentence_transformers import SentenceTransformer
import weaviate.classes as wvc
from weaviate.classes.config import Property, DataType, Configure, VectorDistances
from openai import OpenAI
import weaviate
from weaviate.classes.init import Auth
from weaviate.classes.config import Configure
import os
import json

load_dotenv()

base_url = os.getenv("MODEL_BASE_URL")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")



# Best practice: store your credentials in environment variables
weaviate_url = os.environ["WEAVIATE_URL"]
weaviate_api_key = os.environ["WEAVIATE_API_KEY"]

weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,                                    # Replace with your Weaviate Cloud URL
    auth_credentials=Auth.api_key(weaviate_api_key),             # Replace with your Weaviate Cloud key
)

# OpenAI client setup for Azure
embedding_client = OpenAI(
    api_key=api_key,
    base_url= base_url,
    default_headers={"api-key": api_key},
    default_query={"api-version": api_version}
)

# Embedding function
def get_embedding(text):
    response = embedding_client.embeddings.create(input=text, model="text-embedding-ada-002")
    return response.data[0].embedding 


# OpenAI client setup for Azure
embedding_client = OpenAI(
    api_key=api_key,
    base_url= base_url,
    default_headers={"api-key": api_key},
    default_query={"api-version": api_version}
)

# Embedding function
def get_embedding(text):
    response = embedding_client.embeddings.create(input=text, model="text-embedding-ada-002")
    return response.data[0].embedding 

# --- Create schema if not exists ---
collection_name = "Docs"

if not weaviate_client.collections.exists(collection_name):
    weaviate_client.collections.create(
        name=collection_name,
        properties=[
            weaviate.classes.Property(name="doc_id", data_type=weaviate.classes.DataType.TEXT),
            weaviate.classes.Property(name="source", data_type=weaviate.classes.DataType.TEXT),
            weaviate.classes.Property(name="last_updated", data_type=weaviate.classes.DataType.TEXT),
        ],
        vectorizer_config=weaviate.classes.Configure.Vectorizer.none()  # because we're supplying vectors
    )

collection = weaviate_client.collections.get(collection_name)

# --- Load data from JSON ---
with open("self_critique_loop_dataset.json", "r") as f:
    records = json.load(f)

if isinstance(records, dict):  # in case it's a single record
    records = [records]

# --- Batch insert with embeddings ---
with collection.batch.dynamic() as batch:
    for item in records:        
        combined_text = f"{item['question']} {item['answer_snippet']}"
        vector = get_embedding(combined_text)
        metadata = { "doc_id" : item.get("doc_id"),
            "source": item.get("source"),
            "last_updated": item.get("last_updated")
        }
        batch.add_object(vector=vector, properties=metadata)

print("✅ All records embedded and uploaded to Weaviate.")


In [8]:
import os
import json
from dotenv import load_dotenv
import weaviate
from weaviate.classes.init import Auth
from weaviate.classes.config import Property, DataType, Configure
from openai import OpenAI

load_dotenv()

base_url = os.getenv("MODEL_BASE_URL")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

weaviate_url = os.environ["WEAVIATE_URL"]
weaviate_api_key = os.environ["WEAVIATE_API_KEY"]

# Connect to Weaviate
weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

# OpenAI embedding client for Azure
embedding_client = OpenAI(
    api_key=api_key,
    base_url=base_url,
    default_headers={"api-key": api_key},
    default_query={"api-version": api_version}
)

def get_embedding(text):
    response = embedding_client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding

# --- Create schema if not exists ---
collection_name = "Docs"
if not weaviate_client.collections.exists(collection_name):
    weaviate_client.collections.create(
        name=collection_name,
        properties=[
            Property(name="doc_id", data_type=DataType.TEXT),
            Property(name="answer_snippet", data_type=DataType.TEXT),    # <--- Add this!
            Property(name="source", data_type=DataType.TEXT),
            Property(name="last_updated", data_type=DataType.TEXT),
        ],
        vectorizer_config=Configure.Vectorizer.none()
    )

collection = weaviate_client.collections.get(collection_name)

# --- Load data from JSON ---
with open("self_critique_loop_dataset.json", "r") as f:
    records = json.load(f)

if isinstance(records, dict):
    records = [records]

# --- Batch insert with embeddings ---
with collection.batch.dynamic() as batch:
    for item in records:
        combined_text = f"{item.get('question', '')} {item.get('answer_snippet', '')}"
        vector = get_embedding(combined_text)
        metadata = {
            "doc_id": item.get("doc_id"),
            "answer_snippet": item.get("answer_snippet"),    # <--- Add this!
            "source": item.get("source"),
            "last_updated": item.get("last_updated")
        }
        batch.add_object(vector=vector, properties=metadata)

print("✅ All records (with snippets) embedded and uploaded to Weaviate.")


/home/user/gen-ai/lib/python3.12/site-packages/weaviate/warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
/tmp/ipykernel_6205/2319361493.py:63: ResourceWarning: unclosed <ssl.SSLSocket fd=68, family=2, type=1, proto=6, laddr=('10.33.191.12', 47696), raddr=('34.160.174.119', 443)>
  with collection.batch.dynamic() as batch:


✅ All records (with snippets) embedded and uploaded to Weaviate.
